## Load Directory Locations

In [1]:
import json
import os

# Check if the file exists and load the JSON file into a dictionary
file_path = r'C:\Users\mike\Develop\Projects\Code Notebook\Credentials\locations_conf.json'
if os.path.exists(file_path):
    with open(file_path, 'r') as f:
        locations_data = json.load(f)
    print(locations_data)
else:
    print(f"File not found: {file_path}")

{'Common_Funcs_Dir': '/Users/mike/Develop/Projects/Code Notebook/Common/Functions', 'Credentials_Dir': '/Users/mike/Develop/Projects/Code Notebook/Credentials', 'Rel_Pickes_Dir': '../.pickles', 'Pub_Data_Dir': "'/Users/mike/Data/Public", 'BQ_Service_Key': '/Users/mike/Develop/Conf/GCP Service Keys/mikecancell-development-0bcca41f8486.json'}


# Load the DF by Querying Grouped Crime Data from GCP

In [2]:
from google.oauth2 import service_account
import pandas_gbq

# Ensure locations_data is defined
if 'locations_data' not in globals():
	# Check if the file exists and load the JSON file into a dictionary
	if os.path.exists(file_path):
		with open(file_path, 'r') as f:
			locations_data = json.load(f)
	else:
		raise FileNotFoundError(f"File not found: {file_path}")

# Resolve the key path from the locations data
key_path = locations_data.get('BQ_Service_Key', 'default_key_path.json')

# Create credentials using the key file
credentials = service_account.Credentials.from_service_account_file(key_path)

# Define your project ID and the SQL query
project_id = 'mikecancell-development'
query = """
SELECT *
FROM `mikecancell-development.Datasets.LA_Crime_By_Mon_Crm_Loc_Age`
"""

# Read the data from BigQuery into a pandas DataFrame
crime_data = pandas_gbq.read_gbq(query, project_id=project_id, credentials=credentials)

# Display the first few rows of the dataframe
print(crime_data.head())

Downloading: 100%|██████████|
  dateyear_occ datemon_occ      lat       lon  area_id vict_age_group  \
0   2019-01-01  2019-08-01  34.2693 -118.4241       16    55 and over   
1   2019-01-01  2019-08-01  34.2629 -118.4481       19    55 and over   
2   2019-01-01  2019-01-01  33.7937 -118.2930        5          35-54   
3   2019-01-01  2019-01-01  34.2012 -118.3790       15       Under 18   
4   2019-01-01  2019-01-01  34.0417 -118.2142        4    55 and over   

   fk_crm_cd  crime_count  avg_crimes_per_day  crime_count_in_month  ...  \
0        331            1                 1.0                 17691  ...   
1        310            1                 1.0                 17691  ...   
2        930            2                 2.0                 17487  ...   
3        850            2                 2.0                 17487  ...   
4        310            1                 1.0                 17487  ...   

   crime_score_by_crmcd  crime_score_by_age_group  \
0                  18